## Raw Data

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import lightgbm as lgb
from sklearn.model_selection import train_test_split

df_princess_plot = {
    "Week": [
        "Oct Wk4", "Oct Wk5", "Nov Wk1", "Nov Wk2", "Nov Wk3", "Nov Wk4",
        "Dec Wk1", "Dec Wk2", "Dec Wk3", "Dec Wk4", "Jan Wk1", "Jan Wk2",
        "Jan Wk3", "Jan Wk4", "Jan Wk5"
    ],
    "AMR":
    [240, 170, 130, 90, 110, 130, 110, 110, 110, 130, 70, 90, 100, 80, 90],
    "Europe": [100, 80, 90, 80, 70, 60, 60, 60, 50, 50, 50, 80, 80, 60, 50],
    "PAC": [
        150, 220, 240, 150, 130, 120, 110, 100, 110, 100, 120, 130, 160, 120,
        100
    ]
}
df_dwarf_plot = {
    "Week": [
        "Sept Wk3", "Sept Wk4", "Oct Wk1", "Oct Wk2", "Oct Wk3", "Oct Wk4",
        "Oct Wk5", "Nov Wk1", "Nov Wk2", "Nov Wk3", "Nov Wk4", "Dec Wk1",
        "Dec Wk2", "Dec Wk3", "Dec Wk4"
    ],
    "AMR": [
        320, 220, 170, 190, 200, 170, 160, 160, 140, 140, 180, 160, 160, 170,
        190
    ],
    "Europe": [80, 100, 60, 100, 100, 90, 80, 80, 80, 70, 90, 80, 80, 80, 70],
    "PAC":
    [230, 210, 140, 140, 140, 150, 140, 175, 140, 90, 90, 100, 110, 100, 90]
}

df1 = pd.DataFrame(df_princess_plot)
df2 = pd.DataFrame(df_dwarf_plot)

## Model

### Train

In [ ]:
# prepare data - train and predict
df_model = pd.concat([
    df1.set_index('Week').unstack().rename('y').reset_index().rename(
        columns={
            'level_0': 'Region'
        }).assign(price=200, week_distance=list(range(15)) * 3),
    df2.set_index('Week').unstack().rename('y').reset_index().rename(
        columns={
            'level_0': 'Region'
        }).assign(price=120, week_distance=list(range(15)) * 3),
    df2.set_index('Week').unstack().rename('y').reset_index().rename(
        columns={
            'level_0': 'Region'
        }).assign(price=205, week_distance=list(range(15)) * 3).drop(
            columns='y')  # use dwarf's data format
])

In [ ]:
# add potential holidays
df_model.loc[(df_model.Region == 'PAC') &
             (df_model.Week.isin(['Nov Wk1', 'Jan Wk3'])), 'holiday'] = 1
df_model.loc[(df_model.Region == 'AMR') &
             (df_model.Week.isin(['Nov Wk4', 'Dec Wk4'])), 'holiday'] = 1
df_model.loc[(df_model.Region == 'Europe') & (df_model.Week.isin(['Nov Wk4'])),
             'holiday'] = 1
df_model['holiday'].fillna(0, inplace=True)
df_model[['Region', 'Week',
          'holiday']] = df_model[['Region', 'Week',
                                  'holiday']].astype('category')

# add initial week signal
df_model.loc[df_model.week_distance == 0, 'initial_week'] = 1
df_model['initial_week'].fillna(0, inplace=True)

# X and y
X = df_model[~df_model.y.isnull()].drop(columns='y')
y = df_model[~df_model.y.isnull()]['y']

# split to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# params
params = {
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': ['mae', 'rmse'],
    'min_data_in_leaf': 1,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'force_col_wise': True,
    'random_state': 42
}

# train
model = lgb.train(
    params,
    train_data,
    num_boost_round=500,
    valid_sets=[train_data, test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=30),
               lgb.log_evaluation(50)])

### Predict

In [ ]:
df_model.loc[df_model.y.isnull(),
             'y'] = model.predict(df_model[df_model.y.isnull()].drop(
                 columns='y')).round()  # unit should be int

df3 = df_model[df_model.price == 205].pivot_table(
    index=['Week', 'week_distance'],
    columns='Region',
    values='y',
    aggfunc='max',
    dropna=True).reset_index().sort_values(
        by='week_distance', ignore_index=True).drop(columns='week_distance')

### Feature importance check

In [ ]:
lgb.plot_importance(model)

## Visualization

In [ ]:
# data
data = {
    "Chart1": {
        "df": df1,
        "line_style": "solid"
    },
    "Chart2": {
        "df": df2,
        "line_style": "solid"
    },
    "Chart3": {
        "df": df3,
        "line_style": "dash"
    }
}

# color palette - Tableau
region_colors = {
    "AMR": "#4E79A7",  # blue
    "Europe": "#E15759",  # red
    "PAC": "#59A14F"  # gren
}

# layout
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=("Princess Plus", "", "Dwarf Plus", "", "Superman Plus Estimate", ""),
    column_widths=[0.7, 0.3],
    specs=[
        [{"type": "scatter"}, {"type": "table"}],
        [{"type": "scatter"}, {"type": "table"}],
        [{"type": "scatter"}, {"type": "table"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.05
)

# add plots
for row, (key, item) in enumerate(data.items(), start=1):
    df = item["df"]
    
    # line chart
    for col in df.columns[1:]:
        fig.add_trace(
            go.Scatter(
                x=df["Week"],
                y=df[col],
                name=col if row==1 else "",
                legendgroup=col,
                line=dict(
                    color=region_colors[col],
                    width=2.5,
                    dash=item["line_style"]
                ),
                showlegend=row==1,
                hovertemplate="Week: %{x}<br>Demand: %{y}",
                marker=dict(size=6)
            ),
            row=row, col=1
        )
        fig.update_yaxes(range=[30, 350])
    fig.update_xaxes(title_text="Week", row=row, col=1)
    fig.update_yaxes(title_text="Demand", row=row, col=1)
    
    
    # table
    fig.add_trace(
        go.Table(
            header=dict(
                values=df.columns,
                align="center",
                font=dict(size=13, color="white"),
                fill_color="#4E79A7",  # header color
                line_color="darkslategray"
            ),
            cells=dict(
                values=[df[col] for col in df.columns],
                align="center",
                font=dict(size=12),
                fill_color=["white", "#F7F7F7"],  # banded color
                line_color="lightgray"
            ),
            columnorder=[0,1,2,3]
        ),
        row=row, col=2
    )


# optimize the layout
fig.update_layout(
    height=1200,
    title_text="<b>Regional Demand Analysis</b>",
    title_x=0.5,
    title_font=dict(size=18, family="Arial"),
    hovermode="x unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        bgcolor="rgba(255,255,255,0.7)"
    ),
    plot_bgcolor="rgba(240,240,240,0.5)",  # light gray background
    paper_bgcolor="white",
)
fig.show()

# standardize the axes
for row in range(1,4):
    fig.update_xaxes(
        title_text="Week",
        row=row, col=1,
        linecolor="lightgray",
        gridcolor="rgba(211,211,211,0.3)"
    )
    fig.update_yaxes(
        title_text="Demand (Units)",
        row=row, col=1,
        linecolor="lightgray",
        gridcolor="rgba(211,211,211,0.3)",
    )

    
with open("forecast_dashboard.html", "w") as f:
    f.write("""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            .plot-container {
                width: 90%;
                margin: 0 auto;
                padding: 0px;
            }
        </style>
    </head>
    <body>
        <div class="plot-container">
    """)
    
    fig.write_html(
        f,
        full_html=False,
        include_plotlyjs='cdn',config={'responsive': True}
    )
    
    f.write("""
        </div>
    </body>
    </html>
    """)